In [5]:
import json
# This is necessary to recognize the modules
import os
import sys
import datetime
import warnings

warnings.filterwarnings("ignore")

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [6]:
from core.backtesting import BacktestingEngine

backtesting = BacktestingEngine(root_path=root_path, load_cached_data=True)

In [8]:
from controllers.generic.dneitor import DneitorConfig, GridRange
from decimal import Decimal


# Controller configuration
connector_name = "binance"
trading_pair = "WLD-USDT"
total_amount_quote = 500
prices = [3, 1.9, 1.4]
amounts_quote_pct = [0.5, 0.5, 2]
activation_bounds = 0.03
max_open_orders = 200
start = int(datetime.datetime(2024, 9, 22).timestamp())
end = int(datetime.datetime(2024, 9, 24).timestamp())
backtesting_resolution = "1m"


# Creating the instance of the configuration and the controller
config = DneitorConfig(
    connector_name=connector_name,
    trading_pair=trading_pair,
    total_amount_quote=Decimal(total_amount_quote),
    grid_ranges=[
        GridRange(id="R1", start_price=Decimal("2.0"), end_price=Decimal("2.5"), total_amount_pct=Decimal("0.5")),
        GridRange(id="R2", start_price=Decimal("1.0"), end_price=Decimal("1.5"), total_amount_pct=Decimal("0.5")),
    ],
    activation_bounds=activation_bounds,
    max_open_orders=max_open_orders)

In [13]:
from core.utils import dump_dict_to_yaml
import json

dump_dict_to_yaml(json.loads(config.json()), "configs")

In [9]:
from research_notebooks.dneitor.dneitor_bt import DneitorBacktesting

backtester = DneitorBacktesting()

# Running the backtesting this will output a backtesting result object that has built in methods to visualize the results
backtesting_result = await backtesting.run_backtesting(
    config=config,
    start=start,
    end=end,
    backtesting_resolution=backtesting_resolution,
    backtester=backtester
)

2024-09-24 22:23:35,964 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x14b3a7010>
2024-09-24 22:23:35,965 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x1684d62c0>, 3992.908164833)]']
connector: <aiohttp.connector.TCPConnector object at 0x14b3a7d30>
2024-09-24 22:23:36,911 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x14b4a5960>
2024-09-24 22:23:36,912 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x14b3f0c70>
2024-09-24 22:23:37,491 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x16817c2b0>
2024-09-24 22:23:37,495 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x14c25eec0>, 3994.414965041)]']
connector: <aiohttp.connector.TCPConnector object at 0x16817d9c0

In [10]:
# Let's see what is inside the backtesting results
print(backtesting_result.get_results_summary())
fig = backtesting_result.get_backtesting_figure()
for grid_level in backtester.controller.grid_levels:
    fig.add_hline(y=grid_level.price, line_dash="dot", line_color="gray")
fig.show()


Net PNL: $25.56 (5.11%) | Max Drawdown: $-3.79 (-0.76%)
Total Volume ($): 6855.13 | Sharpe Ratio: 4.61 | Profit Factor: 7.75
Total Executors: 741 | Accuracy Long: 0.93 | Accuracy Short: 0.00
Close Types: Take Profit: 0 | Stop Loss: 0 | Time Limit: 50 |
             Trailing Stop: 650 | Early Stop: 41



In [ ]:
# 2. The executors dataframe: this is the dataframe that contains the information of the orders that were executed
import pandas as pd

executors_df = backtesting_result.executors_df
executors_df.head()

### Backtesting Analysis

### Scatter of PNL per Trade
This bar chart illustrates the PNL for each individual trade. Positive PNLs are shown in green and negative PNLs in red, providing a clear view of profitable vs. unprofitable trades.


In [ ]:
import plotly.express as px

# Create a new column for profitability
executors_df['profitable'] = executors_df['net_pnl_quote'] > 0

# Create the scatter plot
fig = px.scatter(
    executors_df,
    x="timestamp",
    y='net_pnl_quote',
    title='PNL per Trade',
    color='profitable',
    color_discrete_map={True: 'green', False: 'red'},
    labels={'timestamp': 'Timestamp', 'net_pnl_quote': 'Net PNL (Quote)'},
    hover_data=['filled_amount_quote', 'side']
)

# Customize the layout
fig.update_layout(
    xaxis_title="Timestamp",
    yaxis_title="Net PNL (Quote)",
    legend_title="Profitable",
    font=dict(size=12, color="white"),
    showlegend=False,
    plot_bgcolor='rgba(0,0,0,0.8)',  # Dark background
    paper_bgcolor='rgba(0,0,0,0.8)',  # Dark background for the entire plot area
    xaxis=dict(gridcolor="gray"),
    yaxis=dict(gridcolor="gray")
)

# Add a horizontal line at y=0 to clearly separate profits and losses
fig.add_hline(y=0, line_dash="dash", line_color="lightgray")

# Show the plot
fig.show()

### Histogram of PNL Distribution
The histogram displays the distribution of PNL values across all trades. It helps in understanding the frequency and range of profit and loss outcomes.


In [ ]:
fig = px.histogram(executors_df, x='net_pnl_quote', title='PNL Distribution')
fig.show()


# Conclusion
We can see that the indicator has potential to bring good signals to trade and might be interesting to see how we can design a market maker that shifts the mid price based on this indicator.
A lot of the short signals are wrong but if we zoom in into the loss signals we can see that the losses are not that big and the wins are bigger and if we had implemented the trailing stop feature probably a lot of them are going to be profits.

# Next steps
- Filter only the loss signals and understand what you can do to prevent them
- Try different configuration values for the indicator
- Test in multiple markets, pick mature markets like BTC-USDT or ETH-USDT and also volatile markets like DOGE-USDT or SHIB-USDT